# DuckDB Analytics Examples

This notebook demonstrates high-performance analytics using DuckDB, an in-process analytical database optimized for complex queries and large dataset analysis.

In [ ]:
// Import required packages for DuckDB analytics
#r "nuget: DuckDB.NET.Data.Full, 1.1.3"
#r "nuget: Microsoft.Data.Analysis, 0.22.0"
#r "nuget: Plotly.NET, 5.0.0"
#r "nuget: Plotly.NET.Interactive, 5.0.0"
#r "nuget: System.Text.Json, 9.0.0"

using DuckDB.NET.Data;
using Microsoft.Data.Analysis;
using Plotly.NET;
using Plotly.NET.Interactive;
using System.Text.Json;
using System.Data;

// Configure DuckDB connection
var connectionString = NotebookConfiguration.DuckDB.GetConnectionString();
using var connection = new DuckDBConnection(connectionString);
await connection.OpenAsync();

Console.WriteLine("🦆 DuckDB Analytics Engine Ready!");
Console.WriteLine($"Connection: {(connectionString == ":memory:" ? "In-Memory Database" : connectionString)}");

// Enable DuckDB extensions for enhanced analytics
await connection.ExecuteAsync("INSTALL httpfs; LOAD httpfs;"); // For remote file access
Console.WriteLine("✅ Extensions loaded: httpfs for remote data access");

## Sample Data Creation

Generate sample datasets for analytics demonstrations.

In [ ]:
// Create sample sales data
await connection.ExecuteAsync(@"
    CREATE TABLE sales_data (
        order_id INTEGER,
        customer_id INTEGER,
        product_category VARCHAR(50),
        product_name VARCHAR(100),
        quantity INTEGER,
        unit_price DECIMAL(10,2),
        order_date DATE,
        region VARCHAR(20),
        sales_rep VARCHAR(50)
    );");

// Insert sample sales records
await connection.ExecuteAsync(@"
    INSERT INTO sales_data VALUES
    (1001, 501, 'Electronics', 'Laptop', 2, 999.99, '2024-01-15', 'North', 'Alice Johnson'),
    (1002, 502, 'Electronics', 'Smartphone', 1, 699.99, '2024-01-16', 'South', 'Bob Smith'),
    (1003, 503, 'Furniture', 'Office Chair', 4, 149.99, '2024-01-17', 'East', 'Carol Davis'),
    (1004, 504, 'Electronics', 'Tablet', 3, 299.99, '2024-01-18', 'West', 'David Wilson'),
    (1005, 501, 'Furniture', 'Desk', 1, 399.99, '2024-01-19', 'North', 'Alice Johnson'),
    (1006, 505, 'Books', 'Programming Guide', 2, 49.99, '2024-01-20', 'South', 'Eve Brown'),
    (1007, 506, 'Electronics', 'Monitor', 2, 249.99, '2024-01-21', 'East', 'Carol Davis'),
    (1008, 507, 'Furniture', 'Bookshelf', 1, 199.99, '2024-01-22', 'West', 'David Wilson'),
    (1009, 508, 'Books', 'Data Science Handbook', 1, 59.99, '2024-01-23', 'North', 'Alice Johnson'),
    (1010, 509, 'Electronics', 'Headphones', 3, 79.99, '2024-01-24', 'South', 'Bob Smith');");

// Create customer demographics table
await connection.ExecuteAsync(@"
    CREATE TABLE customer_demographics (
        customer_id INTEGER,
        age_group VARCHAR(20),
        gender VARCHAR(10),
        income_bracket VARCHAR(20),
        location_type VARCHAR(15)
    );");

await connection.ExecuteAsync(@"
    INSERT INTO customer_demographics VALUES
    (501, '25-34', 'Female', '50K-75K', 'Urban'),
    (502, '35-44', 'Male', '75K-100K', 'Suburban'),
    (503, '45-54', 'Female', '100K+', 'Urban'),
    (504, '25-34', 'Male', '50K-75K', 'Rural'),
    (505, '35-44', 'Female', '25K-50K', 'Suburban'),
    (506, '18-24', 'Male', '25K-50K', 'Urban'),
    (507, '55+', 'Female', '75K-100K', 'Suburban'),
    (508, '45-54', 'Male', '100K+', 'Urban'),
    (509, '35-44', 'Female', '50K-75K', 'Rural'),
    (510, '25-34', 'Male', '75K-100K', 'Suburban');");

Console.WriteLine("✅ Sample datasets created:");
Console.WriteLine("  - sales_data: Transaction records with products, prices, and dates");
Console.WriteLine("  - customer_demographics: Customer profiles and segments");

// Verify data creation
var salesCount = await connection.QuerySingleAsync<int>("SELECT COUNT(*) FROM sales_data");
var customerCount = await connection.QuerySingleAsync<int>("SELECT COUNT(*) FROM customer_demographics");

Console.WriteLine($"📊 Data loaded: {salesCount} sales records, {customerCount} customer profiles");

## Analytical Queries

Perform complex analytical queries to extract business insights.

In [ ]:
// Sales performance analysis
var salesByCategory = await connection.QueryAsync(@"
    SELECT 
        product_category,
        COUNT(*) as order_count,
        SUM(quantity * unit_price) as total_revenue,
        AVG(quantity * unit_price) as avg_order_value,
        SUM(quantity) as total_units_sold
    FROM sales_data
    GROUP BY product_category
    ORDER BY total_revenue DESC");

Console.WriteLine("💰 Sales Performance by Category:\n");
foreach (var category in salesByCategory)
{
    Console.WriteLine($"{category.product_category}:");
    Console.WriteLine($"  Orders: {category.order_count}");
    Console.WriteLine($"  Revenue: ${category.total_revenue:N2}");
    Console.WriteLine($"  Avg Order: ${category.avg_order_value:N2}");
    Console.WriteLine($"  Units Sold: {category.total_units_sold}");
    Console.WriteLine();
}

// Regional performance analysis
var regionalPerformance = await connection.QueryAsync(@"
    SELECT 
        region,
        COUNT(DISTINCT customer_id) as unique_customers,
        COUNT(*) as total_orders,
        SUM(quantity * unit_price) as total_revenue,
        AVG(quantity * unit_price) as avg_order_value
    FROM sales_data
    GROUP BY region
    ORDER BY total_revenue DESC");

Console.WriteLine("🗺️ Regional Performance Analysis:\n");
foreach (var region in regionalPerformance)
{
    Console.WriteLine($"{region.region} Region:");
    Console.WriteLine($"  Unique Customers: {region.unique_customers}");
    Console.WriteLine($"  Total Orders: {region.total_orders}");
    Console.WriteLine($"  Revenue: ${region.total_revenue:N2}");
    Console.WriteLine($"  Avg Order Value: ${region.avg_order_value:N2}");
    Console.WriteLine();
}

## Customer Analytics

Analyze customer behavior and segmentation patterns.

In [ ]:
// Customer segmentation analysis
var customerSegments = await connection.QueryAsync(@"
    SELECT 
        cd.age_group,
        cd.income_bracket,
        COUNT(DISTINCT s.customer_id) as customer_count,
        COUNT(s.order_id) as total_orders,
        SUM(s.quantity * s.unit_price) as total_spent,
        AVG(s.quantity * s.unit_price) as avg_order_value,
        STRING_AGG(DISTINCT s.product_category, ', ') as preferred_categories
    FROM sales_data s
    JOIN customer_demographics cd ON s.customer_id = cd.customer_id
    GROUP BY cd.age_group, cd.income_bracket
    ORDER BY total_spent DESC");

Console.WriteLine("👥 Customer Segmentation Analysis:\n");
foreach (var segment in customerSegments)
{
    Console.WriteLine($"{segment.age_group} | {segment.income_bracket}:");
    Console.WriteLine($"  Customers: {segment.customer_count}");
    Console.WriteLine($"  Orders: {segment.total_orders}");
    Console.WriteLine($"  Total Spent: ${segment.total_spent:N2}");
    Console.WriteLine($"  Avg Order: ${segment.avg_order_value:N2}");
    Console.WriteLine($"  Categories: {segment.preferred_categories}");
    Console.WriteLine();
}

// Customer lifetime value analysis
var customerLTV = await connection.QueryAsync(@"
    SELECT 
        s.customer_id,
        cd.age_group,
        cd.income_bracket,
        COUNT(s.order_id) as order_frequency,
        SUM(s.quantity * s.unit_price) as total_value,
        AVG(s.quantity * s.unit_price) as avg_order_value,
        MAX(s.order_date) as last_order_date,
        DATE_DIFF('day', MIN(s.order_date), MAX(s.order_date)) as customer_tenure_days
    FROM sales_data s
    JOIN customer_demographics cd ON s.customer_id = cd.customer_id
    GROUP BY s.customer_id, cd.age_group, cd.income_bracket
    ORDER BY total_value DESC");

Console.WriteLine("💎 Customer Lifetime Value Analysis (Top 5):\n");
foreach (var customer in customerLTV.Take(5))
{
    Console.WriteLine($"Customer {customer.customer_id} ({customer.age_group}, {customer.income_bracket}):");
    Console.WriteLine($"  Total Value: ${customer.total_value:N2}");
    Console.WriteLine($"  Order Frequency: {customer.order_frequency}");
    Console.WriteLine($"  Avg Order: ${customer.avg_order_value:N2}");
    Console.WriteLine($"  Tenure: {customer.customer_tenure_days} days");
    Console.WriteLine($"  Last Order: {customer.last_order_date:yyyy-MM-dd}");
    Console.WriteLine();
}

## Time Series Analysis

Analyze trends and patterns over time.

In [ ]:
// Generate additional time series data
await connection.ExecuteAsync(@"
    CREATE TABLE daily_metrics AS
    SELECT 
        order_date,
        COUNT(*) as daily_orders,
        SUM(quantity * unit_price) as daily_revenue,
        AVG(quantity * unit_price) as avg_order_value,
        COUNT(DISTINCT customer_id) as unique_customers
    FROM sales_data
    GROUP BY order_date
    ORDER BY order_date");

// Time series analysis with window functions
var timeSeriesAnalysis = await connection.QueryAsync(@"
    SELECT 
        order_date,
        daily_orders,
        daily_revenue,
        avg_order_value,
        unique_customers,
        LAG(daily_revenue, 1) OVER (ORDER BY order_date) as prev_day_revenue,
        daily_revenue - LAG(daily_revenue, 1) OVER (ORDER BY order_date) as revenue_change,
        AVG(daily_revenue) OVER (
            ORDER BY order_date 
            ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
        ) as three_day_avg_revenue,
        SUM(daily_orders) OVER (
            ORDER BY order_date 
            ROWS UNBOUNDED PRECEDING
        ) as cumulative_orders
    FROM daily_metrics
    ORDER BY order_date");

Console.WriteLine("📈 Time Series Analysis:\n");
foreach (var day in timeSeriesAnalysis)
{
    var changeIndicator = day.revenue_change switch
    {
        > 0 => "📈",
        < 0 => "📉", 
        _ => "➡️"
    };
    
    Console.WriteLine($"{day.order_date:yyyy-MM-dd} {changeIndicator}");
    Console.WriteLine($"  Orders: {day.daily_orders} | Revenue: ${day.daily_revenue:N2}");
    Console.WriteLine($"  Avg Order: ${day.avg_order_value:N2} | Customers: {day.unique_customers}");
    
    if (day.revenue_change.HasValue)
    {
        Console.WriteLine($"  Change: ${day.revenue_change:+0.00;-0.00} | 3-day Avg: ${day.three_day_avg_revenue:N2}");
    }
    
    Console.WriteLine($"  Cumulative Orders: {day.cumulative_orders}");
    Console.WriteLine();
}

## Advanced Analytics

Perform statistical analysis and data mining operations.

In [ ]:
// Statistical analysis using DuckDB's statistical functions
var statisticalSummary = await connection.QuerySingleAsync(@"
    SELECT 
        COUNT(*) as total_orders,
        AVG(quantity * unit_price) as mean_order_value,
        MEDIAN(quantity * unit_price) as median_order_value,
        STDDEV(quantity * unit_price) as stddev_order_value,
        MIN(quantity * unit_price) as min_order_value,
        MAX(quantity * unit_price) as max_order_value,
        PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY quantity * unit_price) as q1_order_value,
        PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY quantity * unit_price) as q3_order_value
    FROM sales_data");

Console.WriteLine("📊 Statistical Summary of Order Values:\n");
Console.WriteLine($"Total Orders: {statisticalSummary.total_orders}");
Console.WriteLine($"Mean: ${statisticalSummary.mean_order_value:N2}");
Console.WriteLine($"Median: ${statisticalSummary.median_order_value:N2}");
Console.WriteLine($"Std Deviation: ${statisticalSummary.stddev_order_value:N2}");
Console.WriteLine($"Range: ${statisticalSummary.min_order_value:N2} - ${statisticalSummary.max_order_value:N2}");
Console.WriteLine($"IQR: ${statisticalSummary.q1_order_value:N2} - ${statisticalSummary.q3_order_value:N2}");

// Correlation analysis between different metrics
var correlationAnalysis = await connection.QueryAsync(@"
    WITH customer_metrics AS (
        SELECT 
            cd.customer_id,
            CASE cd.age_group
                WHEN '18-24' THEN 1
                WHEN '25-34' THEN 2  
                WHEN '35-44' THEN 3
                WHEN '45-54' THEN 4
                WHEN '55+' THEN 5
            END as age_numeric,
            CASE cd.income_bracket
                WHEN '25K-50K' THEN 1
                WHEN '50K-75K' THEN 2
                WHEN '75K-100K' THEN 3
                WHEN '100K+' THEN 4
            END as income_numeric,
            COUNT(s.order_id) as order_count,
            AVG(s.quantity * s.unit_price) as avg_order_value,
            SUM(s.quantity * s.unit_price) as total_spent
        FROM customer_demographics cd
        JOIN sales_data s ON cd.customer_id = s.customer_id
        GROUP BY cd.customer_id, cd.age_group, cd.income_bracket
    )
    SELECT 
        CORR(age_numeric, total_spent) as age_spending_correlation,
        CORR(income_numeric, total_spent) as income_spending_correlation,
        CORR(income_numeric, avg_order_value) as income_order_correlation,
        CORR(order_count, total_spent) as frequency_spending_correlation
    FROM customer_metrics");

Console.WriteLine("\n🔗 Correlation Analysis:");
foreach (var corr in correlationAnalysis)
{
    Console.WriteLine($"Age vs Total Spending: {corr.age_spending_correlation:F3}");
    Console.WriteLine($"Income vs Total Spending: {corr.income_spending_correlation:F3}");
    Console.WriteLine($"Income vs Avg Order: {corr.income_order_correlation:F3}");
    Console.WriteLine($"Order Frequency vs Spending: {corr.frequency_spending_correlation:F3}");
}

## Data Visualization

Create interactive charts and visualizations using Plotly.NET.

In [ ]:
// Prepare data for visualization
var categoryData = await connection.QueryAsync(@"
    SELECT 
        product_category,
        SUM(quantity * unit_price) as total_revenue
    FROM sales_data
    GROUP BY product_category
    ORDER BY total_revenue DESC");

// Create bar chart for category performance
var categoryChart = Chart2D.Chart.Column<string, double, string>(
    categoryData.Select(x => x.product_category),
    categoryData.Select(x => (double)x.total_revenue))
    .WithTitle("Revenue by Product Category")
    .WithXAxisStyle(Title.init("Product Category"))
    .WithYAxisStyle(Title.init("Total Revenue ($)"));

categoryChart.Display();

// Prepare regional data
var regionData = await connection.QueryAsync(@"
    SELECT 
        region,
        COUNT(*) as order_count,
        SUM(quantity * unit_price) as total_revenue
    FROM sales_data
    GROUP BY region");

// Create pie chart for regional distribution
var regionChart = Chart2D.Chart.Pie<double, string>(
    regionData.Select(x => (double)x.total_revenue),
    regionData.Select(x => x.region))
    .WithTitle("Revenue Distribution by Region");

regionChart.Display();

// Time series visualization
var timeData = await connection.QueryAsync(@"
    SELECT 
        order_date,
        SUM(quantity * unit_price) as daily_revenue
    FROM sales_data
    GROUP BY order_date
    ORDER BY order_date");

var timeSeriesChart = Chart2D.Chart.Line<DateTime, double, string>(
    timeData.Select(x => x.order_date),
    timeData.Select(x => (double)x.daily_revenue))
    .WithTitle("Daily Revenue Trend")
    .WithXAxisStyle(Title.init("Date"))
    .WithYAxisStyle(Title.init("Revenue ($)"));

timeSeriesChart.Display();

Console.WriteLine("📊 Interactive charts generated:");
Console.WriteLine("✅ Category performance bar chart");
Console.WriteLine("✅ Regional distribution pie chart");
Console.WriteLine("✅ Daily revenue trend line chart");

## Data Export & Reporting

Export analysis results and generate reports.

In [ ]:
// Create comprehensive analytics view
await connection.ExecuteAsync(@"
    CREATE VIEW analytics_summary AS
    SELECT 
        'Sales Overview' as metric_category,
        'Total Orders' as metric_name,
        CAST(COUNT(*) AS VARCHAR) as metric_value,
        'count' as metric_unit
    FROM sales_data
    UNION ALL
    SELECT 
        'Sales Overview' as metric_category,
        'Total Revenue' as metric_name,
        CONCAT('$', FORMAT(SUM(quantity * unit_price), ',.2f')) as metric_value,
        'currency' as metric_unit
    FROM sales_data
    UNION ALL
    SELECT 
        'Sales Overview' as metric_category,
        'Average Order Value' as metric_name,
        CONCAT('$', FORMAT(AVG(quantity * unit_price), ',.2f')) as metric_value,
        'currency' as metric_unit
    FROM sales_data
    UNION ALL
    SELECT 
        'Customer Metrics' as metric_category,
        'Unique Customers' as metric_name,
        CAST(COUNT(DISTINCT customer_id) AS VARCHAR) as metric_value,
        'count' as metric_unit
    FROM sales_data
    UNION ALL
    SELECT 
        'Product Metrics' as metric_category,
        'Top Category' as metric_name,
        (SELECT product_category FROM sales_data 
         GROUP BY product_category 
         ORDER BY SUM(quantity * unit_price) DESC 
         LIMIT 1) as metric_value,
        'category' as metric_unit
    FROM sales_data
    LIMIT 1");

// Generate summary report
var summaryReport = await connection.QueryAsync(@"
    SELECT 
        metric_category,
        metric_name,
        metric_value,
        metric_unit
    FROM analytics_summary
    ORDER BY metric_category, metric_name");

Console.WriteLine("📋 Analytics Summary Report:\n");
string currentCategory = "";
foreach (var metric in summaryReport)
{
    if (metric.metric_category != currentCategory)
    {
        currentCategory = metric.metric_category;
        Console.WriteLine($"\n🏷️ {currentCategory}:");
    }
    Console.WriteLine($"  {metric.metric_name}: {metric.metric_value}");
}

// Export detailed data (simulate CSV export)
var detailedExport = await connection.QueryAsync(@"
    SELECT 
        s.order_id,
        s.customer_id,
        cd.age_group,
        cd.income_bracket,
        s.product_category,
        s.product_name,
        s.quantity,
        s.unit_price,
        (s.quantity * s.unit_price) as total_value,
        s.order_date,
        s.region,
        s.sales_rep
    FROM sales_data s
    JOIN customer_demographics cd ON s.customer_id = cd.customer_id
    ORDER BY s.order_date DESC");

Console.WriteLine($"\n💾 Export prepared: {detailedExport.Count()} records ready for CSV export");
Console.WriteLine("Sample export data (first 3 rows):");

foreach (var row in detailedExport.Take(3))
{
    Console.WriteLine($"  Order {row.order_id}: {row.product_name} | ${row.total_value} | {row.customer_id} ({row.age_group})");
}

## Performance Testing

Test DuckDB's performance with larger datasets and complex queries.

In [ ]:
// Generate larger dataset for performance testing
var stopwatch = System.Diagnostics.Stopwatch.StartNew();

await connection.ExecuteAsync(@"
    CREATE TABLE large_sales_data AS
    SELECT 
        row_number() OVER() + 10000 as order_id,
        (random() * 1000)::INTEGER + 1 as customer_id,
        CASE (random() * 4)::INTEGER
            WHEN 0 THEN 'Electronics'
            WHEN 1 THEN 'Furniture'  
            WHEN 2 THEN 'Books'
            ELSE 'Clothing'
        END as product_category,
        'Product_' || (row_number() OVER())::VARCHAR as product_name,
        (random() * 10 + 1)::INTEGER as quantity,
        (random() * 500 + 10)::DECIMAL(10,2) as unit_price,
        ('2024-01-01'::DATE + (random() * 365)::INTEGER) as order_date,
        CASE (random() * 4)::INTEGER
            WHEN 0 THEN 'North'
            WHEN 1 THEN 'South'
            WHEN 2 THEN 'East'
            ELSE 'West'
        END as region
    FROM range(50000);"); // Generate 50,000 records

stopwatch.Stop();
var dataGenerationTime = stopwatch.ElapsedMilliseconds;

Console.WriteLine($"🏗️ Generated 50,000 records in {dataGenerationTime}ms");

// Performance benchmark queries
var benchmarks = new[]
{
    new { 
        Name = "Aggregation Query", 
        SQL = @"SELECT product_category, COUNT(*), SUM(quantity * unit_price) 
                FROM large_sales_data 
                GROUP BY product_category" 
    },
    new { 
        Name = "Window Function Query", 
        SQL = @"SELECT *, 
                ROW_NUMBER() OVER (PARTITION BY product_category ORDER BY quantity * unit_price DESC) as rank
                FROM large_sales_data 
                LIMIT 1000" 
    },
    new { 
        Name = "Complex Join Query", 
        SQL = @"SELECT l.product_category, COUNT(*) as large_orders, COUNT(s.order_id) as small_orders
                FROM large_sales_data l
                LEFT JOIN sales_data s ON l.product_category = s.product_category
                WHERE l.quantity * l.unit_price > 100
                GROUP BY l.product_category" 
    }
};

Console.WriteLine("\n⚡ Performance Benchmarks:");
foreach (var benchmark in benchmarks)
{
    stopwatch.Restart();
    var result = await connection.QueryAsync(benchmark.SQL);
    stopwatch.Stop();
    
    Console.WriteLine($"  {benchmark.Name}: {stopwatch.ElapsedMilliseconds}ms ({result.Count()} rows)");
}

// Memory usage and final statistics
var finalStats = await connection.QuerySingleAsync(@"
    SELECT 
        (SELECT COUNT(*) FROM sales_data) as original_records,
        (SELECT COUNT(*) FROM large_sales_data) as generated_records,
        (SELECT COUNT(*) FROM sales_data) + (SELECT COUNT(*) FROM large_sales_data) as total_records");

Console.WriteLine($"\n📊 Final Dataset Statistics:");
Console.WriteLine($"  Original Records: {finalStats.original_records:N0}");
Console.WriteLine($"  Generated Records: {finalStats.generated_records:N0}");
Console.WriteLine($"  Total Records: {finalStats.total_records:N0}");

## Cleanup & Summary

Clean up resources and summarize the DuckDB analytics workflow.

In [ ]:
// Final summary and cleanup
Console.WriteLine("🎉 DuckDB Analytics Workflow Complete!\n");
Console.WriteLine("What we accomplished:");
Console.WriteLine("✅ Set up high-performance in-memory analytical database");
Console.WriteLine("✅ Created and populated sample datasets");
Console.WriteLine("✅ Performed complex analytical queries and aggregations");
Console.WriteLine("✅ Conducted customer segmentation and lifetime value analysis");
Console.WriteLine("✅ Executed time series analysis with window functions");
Console.WriteLine("✅ Performed statistical analysis and correlation studies");
Console.WriteLine("✅ Generated interactive visualizations with Plotly.NET");
Console.WriteLine("✅ Created comprehensive analytics reports");
Console.WriteLine("✅ Benchmarked performance with large datasets (50K+ records)");
Console.WriteLine();
Console.WriteLine("🔗 Key DuckDB Features Utilized:");
Console.WriteLine("  - In-memory columnar storage for fast analytics");
Console.WriteLine("  - Advanced SQL features (window functions, CTEs, statistical functions)");
Console.WriteLine("  - High-performance aggregations and joins");
Console.WriteLine("  - Built-in statistical and mathematical functions");
Console.WriteLine("  - Efficient handling of large datasets");
Console.WriteLine("  - Integration with .NET data analysis ecosystem");

// Performance summary
var queryCount = await connection.QuerySingleAsync<int>("SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = 'main'");
Console.WriteLine($"\n📈 Session Statistics:");
Console.WriteLine($"  Tables Created: {queryCount}");
Console.WriteLine($"  Total Records Processed: 50,000+");
Console.WriteLine($"  Connection Type: {(connectionString == ":memory:" ? "In-Memory" : "Persistent")}");

Console.WriteLine("\n🧹 Cleanup: All data stored in memory will be cleared when connection closes");
await connection.CloseAsync();
Console.WriteLine("✅ DuckDB connection closed successfully");